In [1]:
from google.cloud import aiplatform
from google.cloud.aiplatform.gapic.schema import predict
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
from google.oauth2 import service_account
from typing import Dict, List, Union

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('../Finalproject/heart.csv')
df.head()

,Age,ChestPainType,RestingBP,Cholesterol,RestingECG,MaxHR,ExerciseAngina,HeartDisease
0,40,ATA,140,289,Normal,172,N,0
1,49,NAP,160,180,Normal,156,N,1
2,37,ATA,130,283,ST,98,N,0
3,48,ASY,138,214,Normal,108,Y,1
4,54,NAP,150,195,Normal,122,N,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1050 entries, 0 to 1049
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   Age             1050 non-null   int64 
 1   ChestPainType   1050 non-null   object
 2   RestingBP       1050 non-null   int64 
 3   Cholesterol     1050 non-null   int64 
 4   RestingECG      1050 non-null   object
 5   MaxHR           1050 non-null   int64 
 6   ExerciseAngina  1050 non-null   object
 7   HeartDisease    1050 non-null   int64 
dtypes: int64(5), object(3)
memory usage: 65.8+ KB


In [4]:
df.shape

(1050, 8)

In [5]:
df.describe()

,Age,RestingBP,Cholesterol,MaxHR,HeartDisease
count,1050.000000,1050.000000,1050.000000,1050.000000,1050.000000
mean,53.679048,132.325714,204.618095,138.081905,0.544762
std,9.405738,18.151954,105.618681,25.619176,0.498230
min,28.000000,0.000000,0.000000,60.000000,0.000000
25%,47.000000,120.000000,180.250000,120.000000,0.000000
50%,54.000000,130.000000,224.500000,140.000000,1.000000
75%,60.000000,140.000000,267.750000,157.750000,1.000000
max,77.000000,200.000000,603.000000,202.000000,1.000000


In [6]:
print(df.columns)

Index(['Age', 'ChestPainType', 'RestingBP', 'Cholesterol', 'RestingECG',
       'MaxHR', 'ExerciseAngina', 'HeartDisease'],
      dtype='object')


In [7]:
key_path = './comp840-project1-bd32a0b12175.json'
credentials = service_account.Credentials.from_service_account_file(key_path)

In [8]:
def predict_custom_trained_model_sample(
    project: str,
    endpoint_id: str,
    instances: Union[Dict, List[Dict]],
    location: str = "us-central1",
    api_endpoint: str = "us-central1-aiplatform.googleapis.com",
):
    """
    `instances` can be either single instance of type dict or a list
    of instances.
    """
    # The AI Platform services require regional API endpoints.
    client_options = {"api_endpoint": api_endpoint}
    # Initialize client that will be used to create and send requests.
    # This client only needs to be created once, and can be reused for multiple requests.
    client = aiplatform.gapic.PredictionServiceClient(client_options=client_options, credentials=credentials)
    # The format of each instance should conform to the deployed model's prediction input schema.
    instances = instances if type(instances) == list else [instances]
    instances = [
        json_format.ParseDict(instance_dict, Value()) for instance_dict in instances
    ]
    print(instances)
    parameters_dict = {}
    parameters = json_format.ParseDict(parameters_dict, Value())
    endpoint = client.endpoint_path(
        project=project, location=location, endpoint=endpoint_id
    )
    response = client.predict(
        endpoint=endpoint, instances=instances
    )
    print("response")
    print(" deployed_model_id:", response.deployed_model_id)
    # The predictions are a google.protobuf.Value representation of the model's predictions.
    predictions = response.predictions
    #print(" prediction:", predictions)
    for prediction in predictions:
        print(" prediction:", dict(prediction))

In [9]:
predict_custom_trained_model_sample(
    project="678033216167",
    endpoint_id="3431932032056295424",
    location="us-central1",
    instances=[{'Age':'40','ChestPainType':'ATA','RestingBP':'120','Cholesterol':'249','RestingECG':'Normal','MaxHR':'125','ExerciseAngina':'N'},
               {'Age':'26','ChestPainType':'NAP','RestingBP':'180','Cholesterol':'211','RestingECG':'ST','MaxHR':'108','ExerciseAngina':'Y'},
               {'Age':'32','ChestPainType':'ASY','RestingBP':'100','Cholesterol':'130','RestingECG':'Normal','MaxHR':'147','ExerciseAngina':'N'}
              ]
)

[struct_value {
  fields {
    key: "Age"
    value {
      string_value: "40"
    }
  }
  fields {
    key: "ChestPainType"
    value {
      string_value: "ATA"
    }
  }
  fields {
    key: "Cholesterol"
    value {
      string_value: "249"
    }
  }
  fields {
    key: "ExerciseAngina"
    value {
      string_value: "N"
    }
  }
  fields {
    key: "MaxHR"
    value {
      string_value: "125"
    }
  }
  fields {
    key: "RestingBP"
    value {
      string_value: "120"
    }
  }
  fields {
    key: "RestingECG"
    value {
      string_value: "Normal"
    }
  }
}
, struct_value {
  fields {
    key: "Age"
    value {
      string_value: "26"
    }
  }
  fields {
    key: "ChestPainType"
    value {
      string_value: "NAP"
    }
  }
  fields {
    key: "Cholesterol"
    value {
      string_value: "211"
    }
  }
  fields {
    key: "ExerciseAngina"
    value {
      string_value: "Y"
    }
  }
  fields {
    key: "MaxHR"
    value {
      string_value: "108"
    }
  }
  fie